In [1]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def replace_words(t):
    replacers = ["<p>", "<br>", "</p>", "</br>", "<br/>", ":", ",", ".", r'\xao', "\n", "(", ")", "/", "-"]
    for i in replacers:
        t = t.replace(i, " ")
    return t.strip().split()

def search_for_keywords(text):
    keywords = {"comput":5, "protein":1, "drug":5, "design":1, "immun":1, "algorithm":1, "machine":1, "deep":1, "therap":1, "molecular":0, "antibod":1}
    counter = 0
    
    wordlist = replace_words(text)
    for word in keywords.keys():
        for idx, tw in enumerate(wordlist):
            if word in tw or word.capitalize() in tw:
                nextword = "" 
                if idx+1 < len(wordlist):
                    nextword = wordlist[idx+1].lower()
                if word == "protein" and (nextword == "design" or nextword == "engineering"):
                    counter += keywords[word]*10
                elif word == "machine" and nextword == "learning":
                    counter += keywords[word]*5
                elif word == "deep" and nextword == "learning":
                    counter += keywords[word]*5
                elif word == "molecular" and (nextword == "engineering" or nextword == "immunology"):
                    counter += 20
                else:
                    counter += keywords[word]
    return counter

def print_sorted_faculty(faculty_dict, top_twenty=True):
    faculty_list = []
    for f, text in faculty_dict.items():
        num_hits = 0
        try:
            num_hits = search_for_keywords(text)
        except (TypeError, IndexError):
            pass
        faculty_list.append((f, num_hits))

    sorted_fac = sorted(faculty_list, key=lambda x: x[1], reverse=True) 
    print(f"Found and scored {len(sorted_fac)} faculty")
    ctr = 0
    for i in sorted_fac:
        print(i[0][0], i[0][1], i[1])
        if ctr == 19 and top_twenty:
            break
        ctr += 1

# Stanford Faculty Analysis

In [4]:
url = "https://med.stanford.edu/biophysics/faculty1.html"
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
profile_divs = soup.findAll('div', {"class": "profile-name"})
stanford_faculty_dict = {}
for i in profile_divs:
    atags = i.findAll('a')
    for j in atags:
        faculty_url = j['href']
        resp = requests.get(faculty_url)
        fac_soup = BeautifulSoup(resp.text, "html.parser")
        
        bio = fac_soup.findAll('div', {"class": "bio content-section"})
        bio_content = [x.findAll('p') for x in bio]
        
        if len(bio_content) == 0:
            bio_content = [""]
            
        bio_content = str(bio_content[0])
        research_tab = fac_soup.findAll('a', {'id': "research-and-scholarship"})
        url_tail = research_tab[0]['href'] if len(research_tab) != 0 else ""
        research_url = faculty_url + url_tail
        
        research_resp = requests.get(research_url)
        research_soup = BeautifulSoup(research_resp.text, "html.parser")
        research = research_soup.findAll('div', {"class": "interests content-section"})
        research_cont = [x.findAll('p') for x in research]
        if len(research_cont) != 0:
            test = str(research_cont[0][0])
            bio_content += test 
        stanford_faculty_dict[(j.text, faculty_url)] = bio_content      
        
        


In [7]:
print_sorted_faculty(stanford_faculty_dict)

Found and scored 109 faculty
Vijay Pande https://med.stanford.edu/profiles/vijay-pande 108
Russ B. Altman https://med.stanford.edu/profiles/russ-altman 77
Ron Dror https://med.stanford.edu/profiles/ron-dror 60
Crystal Mackall https://med.stanford.edu/profiles/crystal-mackall 40
Manish Saggar https://med.stanford.edu/profiles/manish-saggar 40
Grant M. Rotskoff https://med.stanford.edu/profiles/rotskoff 39
Anshul Kundaje https://med.stanford.edu/profiles/anshul-kundaje 35
Soichi Wakatsuki https://med.stanford.edu/profiles/soichi-wakatsuki 34
Possu Huang https://med.stanford.edu/profiles/possu-huang 32
Peter S. Kim https://med.stanford.edu/profiles/peterkim 30
Rhiju Das https://med.stanford.edu/profiles/rhiju-das 29
Michael Levitt https://med.stanford.edu/profiles/michael-levitt 29
Roseanna N. Zia https://med.stanford.edu/profiles/roseanna-zia 27
Axel Brunger https://med.stanford.edu/profiles/axel-brunger 26
Keren Haroush https://med.stanford.edu/profiles/keren-haroush 26
Tom Wandless htt

# MIT Faculty Analysis

In [8]:
url = "https://csbphd.mit.edu/faculty"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
mit_faculty_dict = {}
profile_trs = soup.findAll('tr')
for idx, i in enumerate(profile_trs):
    if idx == 0:
        continue
    name = i.findAll("a")[0]
    link_tail = name["href"].replace("/faculty", "")
    name = name.text
    resp = requests.get(url + link_tail)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    data = fac_soup.findAll("section", {"class":"field field-name-field-research-summary field-type-text-with-summary field-label-above"})
    text = ""
    for i in data:
        text += i.text
    mit_faculty_dict[(name, url+link_tail)] = text

In [10]:
print_sorted_faculty(mit_faculty_dict)

Found and scored 60 faculty
Barzilay, Regina https://csbphd.mit.edu/faculty/regina-barzilay 44
Kulik, Heather J. https://csbphd.mit.edu/faculty/heather-j-kulik 39
Poggio, Tomaso A. https://csbphd.mit.edu/faculty/tomaso-poggio 32
Bathe, Mark https://csbphd.mit.edu/faculty/mark-bathe 31
Stultz, Collin M. https://csbphd.mit.edu/faculty/collin-m-stultz 30
Gifford, David K. https://csbphd.mit.edu/faculty/david-k-gifford 26
Coley, Connor W. https://csbphd.mit.edu/faculty/connor-w-coley 25
Zhang, Bin https://csbphd.mit.edu/faculty/bin-zhang 25
Sontag, David https://csbphd.mit.edu/faculty/david-sontag 22
Burge, Chris https://csbphd.mit.edu/faculty/chris-burge 21
Hemann, Michael T. https://csbphd.mit.edu/faculty/michael-t-hemann 19
Shalek, Alex K. https://csbphd.mit.edu/faculty/alex-k-shalek 18
Wong, Harikesh S. https://csbphd.mit.edu/faculty/harikesh-s-wong 17
Keating, Amy https://csbphd.mit.edu/faculty/amy-keating 16
Fraenkel, Ernest https://csbphd.mit.edu/faculty/ernest-fraenkel 13
Weiss, Ro

# Pitt-CMU Faculty Analysis

In [11]:
url = "https://www.mbsb.pitt.edu/index.php/people/faculty"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [12]:
pitt_faculty_dict = {}
faculty_list = soup.findAll('div', {"class": "nspArt nspCol4"})
for i in faculty_list:
    name = i.findAll('a')[0]
    link_tail = name['href']
    name = name.text
    prefix = url.split("/index.php")[0]
    faculty_link = prefix + link_tail
    resp = requests.get(faculty_link)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    data = fac_soup.findAll("div", {"class":"item-page"})
    text = ""
    for i in data:
        text += i.text
    pitt_faculty_dict[(name, faculty_link)] = text

In [13]:
print_sorted_faculty(pitt_faculty_dict)

Found and scored 43 faculty
Jacob Durrant https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/277-currant 56
Pei Tang https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/188-pei-tang 50
Olexandr Isayev https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/315-oleander-isayev 25
Yan Xu https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/200-yan-xu 25
Jonathan Coleman https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/298-jonathan-coleman 21
Markus Deserno https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/152-markus-deserno 18
Maria Kurnikova https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/169-maria-kurnikova 17
Cheng Zhang https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/296-cheng-zhang 14
Jeffrey Brodsky https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/144-jeffrey-brodsky 13
Patricia Opresko https://www.mbsb.pitt.edu/index.php/site-map/articles/81-faculty/180-

# UCSF (just bioengineering and pharmacology faculty)

### Bioengineering Faculty

In [14]:
url = "https://bts.ucsf.edu/people/faculty"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [15]:
ucsf_faculty_dict = {}
faculty_table = soup.findAll("div", {"class":"person_name"})
for idx, i in enumerate(faculty_table):
    if idx == 0:
        continue
    name = i.findAll("a")
    link = name[0]['href']
    name = name[0].text
    resp = requests.get(link)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    data = fac_soup.findAll("div", {"class":"moscone-column-content-region-inner moscone-content-inner panel-panel-inner"})
    text = ""
    for i in data:
        text += i.text
    
    ucsf_faculty_dict[(name, link)] = text

In [16]:
print_sorted_faculty(ucsf_faculty_dict)

Found and scored 21 faculty
Tanja Kortemme, PhD https://pharmacy.ucsf.edu/tanja-kortemme 188
Kathy Giacomini, PhD https://pharmacy.ucsf.edu/kathy-giacomini 98
Leslie Benet, PhD https://pharmacy.ucsf.edu/leslie-benet 65
Deanna Kroetz, PhD https://pharmacy.ucsf.edu/deanna-kroetz 55
Francis Szoka, PhD https://pharmacy.ucsf.edu/francis-szoka 54
Rada Savic, PhD https://pharmacy.ucsf.edu/rada-savic 49
Sourav Bandyopadhyay, PhD https://pharmacy.ucsf.edu/sourav-bandyopadhyay 46
C. Anthony Hunt, PhD https://pharmacy.ucsf.edu/c-anthony-hunt 43
Andrej Sali, PhD https://pharmacy.ucsf.edu/andrej-sali 43
Tejal Desai, PhD https://pharmacy.ucsf.edu/tejal-desai 42
Ryan Hernandez, PhD https://pharmacy.ucsf.edu/ryan-hernandez 30
Shuvo Roy, PhD https://pharmacy.ucsf.edu/shuvo-roy 26
James Fraser, PhD https://pharmacy.ucsf.edu/james-fraser 24
Seth Shipman, PhD https://pharmacy.ucsf.edu/seth-shipman 23
Nancy Sambol, PharmD https://pharmacy.ucsf.edu/nancy-sambol 22
Esteban G. Burchard, MD, MPH https://pharma

### Pharmaceutical Chemistry

In [17]:
url = "https://pharmchem.ucsf.edu/people/faculty"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
ucsf_pharm_faculty_dict = {}
faculty_table = soup.findAll("div", {"class":"person_name"})
for idx, i in enumerate(faculty_table):
    if idx == 0:
        continue
    name = i.findAll("a")
    link = name[0]['href']
    name = name[0].text
    resp = requests.get(link)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    data = fac_soup.findAll("div", {"class":"moscone-column-content-region-inner moscone-content-inner panel-panel-inner"})
    text = ""
    for i in data:
        text += i.text
    
    ucsf_pharm_faculty_dict[(name, link)] = text

In [18]:
print_sorted_faculty(ucsf_pharm_faculty_dict)

Found and scored 33 faculty
Michael Keiser, PhD https://pharmacy.ucsf.edu/michael-keiser 82
Thomas James, PhD https://pharmacy.ucsf.edu/thomas-james 70
Brian Shoichet, PhD https://pharmacy.ucsf.edu/brian-shoichet 70
Lei Wang, PhD https://pharmacy.ucsf.edu/lei-wang 62
Tom Ferrin, PhD https://pharmacy.ucsf.edu/tom-ferrin 61
Matt Jacobson, PhD https://pharmacy.ucsf.edu/matt-jacobson 61
William DeGrado, PhD https://pharmacy.ucsf.edu/william-degrado 55
Xiaokun Shu, PhD https://pharmacy.ucsf.edu/xiaokun-shu 53
Michelle Arkin, PhD https://pharmacy.ucsf.edu/michelle-arkin 47
Jim Wells, PhD https://pharmacy.ucsf.edu/jim-wells 44
Paul Ortiz de Montellano, PhD https://pharmacy.ucsf.edu/paul-ortiz-de-montellano 43
Charles S. Craik, PhD https://pharmacy.ucsf.edu/charles-s-craik 40
Lani Wu, PhD https://pharmacy.ucsf.edu/lani-wu 39
Jason Gestwicki, PhD https://pharmacy.ucsf.edu/jason-gestwicki 34
Aashish Manglik, MD, PhD https://pharmacy.ucsf.edu/aashish-manglik 33
Adam Renslo, PhD https://pharmacy.u

# UPenn Faculty Analysis
### All of the faculty have different website formats so this just scrapes everything. Much less useful rankings...

In [19]:
url = "https://www.med.upenn.edu/bmbgrad/bmb_faculty.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [20]:
all_content = soup.findAll("div", {"class":"row main"})
found = False
list_ = all_content[0].findAll("strong")[7]
links = list_.find_all_next("a")
penn_faculty_dict = {}
for i in links:
    try:
        link = i['href']
        name = i.text
        if name == ".":
            continue 
            
        resp = requests.get(link)
        fac_soup = BeautifulSoup(resp.text, "html.parser")
        text = ""
        for i in fac_soup:
            text += i.text
        penn_faculty_dict[(name, link)] = text
        if "Zhaolan" in name:
            break
    except KeyError:
        pass

In [21]:
print_sorted_faculty(penn_faculty_dict)

Found and scored 104 faculty

Luca Busino, Ph.D https://www.med.upenn.edu/cbio/faculty/#personnel-description5 245
Liling Wan, Ph.D. https://www.med.upenn.edu/cbio/faculty/ 245

Daniel Kulp, Ph.D., https://wistar.org/our-scientists/daniel-kulp 118

Nikolaos Sgourakis, Ph.D  https://www.research.chop.edu/mechanistic-molecular-immunology-lab 104

Cesar de la Fuente, Ph.D https://www.seas.upenn.edu/directory/profile.php?ID=299 75

Scott L. Diamond, Ph.D. http://www.seas.upenn.edu/directory/profile.php?ID=24 75

Amish Patel, Ph.D. https://www.seas.upenn.edu/directory/profile.php?ID=182 75

Ravi Radhakrishnan, Ph.D. http://www.seas.upenn.edu/directory/profile.php?ID=76 75
Kai Wang, Ph.D. http://pathology.med.upenn.edu/department/people/1160/kai-wang 75

Arjun Raj, Ph.D. http://www.seas.upenn.edu/be/dir/details/ArjunRajdetails.html 55

Sydney Shaffer, MD, Ph.D. http://pathology.med.upenn.edu/department/people 55

James Shorter, Ph.D. http://www.med.upenn.edu/apps/faculty/index.php/g20x000321

# Harvard Faculty Analysis (Biophysics)

In [22]:
url =  f"https://biophysics.fas.harvard.edu/people/people-terms/faculty"
harvard_faculty_dict = {}
for i in range(0, 6):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    link_list = soup.findAll("div", {"class":"view-content"})[0].findAll("a")
    for link in link_list:
        if "Read more" in link.text:
            name = link.text.split("about")[-1]
            link = link['href']
            
            resp = requests.get(link)
            fac_soup = BeautifulSoup(resp.text, "html.parser")
            text = ""
            for word in fac_soup:
                text += word.text
            harvard_faculty_dict[(name, link)] = text
    url = f"https://biophysics.fas.harvard.edu/people/people-terms/faculty?page={i+1}"

In [23]:
print_sorted_faculty(harvard_faculty_dict)

Found and scored 85 faculty
 Peng Yin https://biophysics.fas.harvard.edu/people/peng-yin 116
 Jan Drugowitsch https://biophysics.fas.harvard.edu/people/jan-drugowitsch 115
 Eugene I. Shakhnovich https://biophysics.fas.harvard.edu/people/eugene-i-shakhnovich 80
 Adam E. Cohen https://biophysics.fas.harvard.edu/people/adam-cohen 50
 Conor L. Evans https://biophysics.fas.harvard.edu/people/conor-l-evans 44
 Andrew C. Kruse https://biophysics.fas.harvard.edu/people/andrew-c-kruse 44
 Chris Sander https://biophysics.fas.harvard.edu/people/chris-sander 41
 Shamil R. Sunyaev https://biophysics.fas.harvard.edu/people/shamil-r-sunyaev 41
 Wei-Chung Allen Lee https://biophysics.fas.harvard.edu/people/wei-chung-allen-lee 40
 Gary Ruvkun https://biophysics.fas.harvard.edu/people/gary-ruvkun 39
 Maofu Liao https://biophysics.fas.harvard.edu/people/maofu-liao 38
 Pamela  A.  Silver https://biophysics.fas.harvard.edu/people/pamela-silver 38
 James J. Chou https://biophysics.fas.harvard.edu/people/jam

# Columbia Faculty Analysis (Doesn't have anything)
### Molecular Pharmacology and Therapeutics

In [24]:
url = "https://www.gsas.cuimc.columbia.edu/faculty-6"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
fac_list = soup.findAll("div", {"class", "field field-name-body field-type-text-with-summary field-label-hidden"})[0]
columbia_faculty_dict = {}
for i in fac_list.findAll("a"):
    fac_url = "https://www.gsas.cuimc.columbia.edu"
    if "mailto" in i['href']:
        continue
    if "https://" in i['href']:
        fac_url = i['href']
    else:
        fac_url = fac_url + i['href']
        
    resp = requests.get(fac_url)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    text = ""
    for word in fac_soup:
        text += word.text
    columbia_faculty_dict[(i.text, fac_url)] = text

In [25]:
print_sorted_faculty(columbia_faculty_dict)

Found and scored 38 faculty
Robert Kass, PhD https://www.gsas.cuimc.columbia.edu/profile/robert-s-kass-phd 70
Kenneth Olive, PhD https://www.gsas.cuimc.columbia.edu/profile/kenneth-p-olive-phd 67
Christian Schindler, MD, PhD https://www.gsas.cuimc.columbia.edu/profile/christian-w-schindler-md 38
Henry Colecraft, PhD https://www.gsas.cuimc.columbia.edu/profile/henry-colecraft-phd 36
Joseph Graziano, PhD https://www.gsas.cuimc.columbia.edu/profile/joseph-h-graziano-phd 26
Andrea Califano, PhD https://www.gsas.cuimc.columbia.edu/profile/andrea-califano-dr 25
Wayne Hendrickson, PhD https://www.gsas.cuimc.columbia.edu/profile/wayne-hendrickson-phd 25
Jonathan Javitch, MD, PhD https://www.gsas.cuimc.columbia.edu/profile/jonathan-javitch-md 19
David Sulzer, PhD https://www.gsas.cuimc.columbia.edu/profile/david-sulzer-phd 19
Virginia Cornish, PhD https://www.gsas.cuimc.columbia.edu/profile/virginia-w-cornish-phd 17
Dritan Agalliu, PhD https://www.gsas.cuimc.columbia.edu/profile/dritan-agalliu-

### Physiology and Cellular Biophysics

In [26]:
url = "https://www.gsas.cuimc.columbia.edu/faculty-7"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
fac_list = soup.findAll("div", {"class", "field field-name-body field-type-text-with-summary field-label-hidden"})[0]
biophysics_columbia_faculty_dict = {}
for i in fac_list.findAll("a"):
    fac_url = "https://www.gsas.cuimc.columbia.edu"
    if "mailto" in i['href']:
        continue
    if "https://" in i['href']:
        fac_url = i['href']
    else:
        fac_url = fac_url + i['href']
        
    try:
        resp = requests.get(fac_url)
        fac_soup = BeautifulSoup(resp.text, "html.parser")
        text = ""
        for word in fac_soup:
            text += word.text
        biophysics_columbia_faculty_dict[(i.text, fac_url)] = text
    except:
        pass

In [27]:
print_sorted_faculty(biophysics_columbia_faculty_dict)

Found and scored 55 faculty
Nicholas Tatonetti, PhD https://www.gsas.cuimc.columbia.edu/profile/nicholas-p-tatonetti-phd 141
Robert Kass, PhD https://www.gsas.cuimc.columbia.edu/profile/robert-s-kass-phd 70
Scott Banta, PhD https://cheme.columbia.edu/faculty/scott-banta 68
Larry Abbott, PhD https://www.gsas.cuimc.columbia.edu/profile/larry-abbott-phd 45
Anne Moscona, MD https://microbiology.columbia.edu/faculty-anne-moscona 40
Henry Colecraft, PhD https://www.gsas.cuimc.columbia.edu/profile/henry-colecraft-phd 36
Ning Qian, PhD https://www.gsas.cuimc.columbia.edu/profile/ning-qian-phd 36
Allie Obermeyer, PhD https://www.engineering.columbia.edu/faculty/allie-obermeyer 31
John Hunt, PhD https://biology.columbia.edu/people/hunt 30
Wayne Hendrickson, PhD https://www.gsas.cuimc.columbia.edu/profile/wayne-hendrickson-phd 25
Jonathan Javitch, MD, PhD https://www.gsas.cuimc.columbia.edu/profile/jonathan-javitch-md 19
Matthias Quick, PhD https://www.columbiapsychiatry.org/profile/matthias-quic

# Northwestern Faculty Analysis (Not working as well because they inject the faculty content with javascript so its not in the initial HTML)
### Biophysics Department

In [28]:
url = "https://biophysics.northwestern.edu/people/preceptors/index.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
faculty_list = soup.findAll("div", {"class":"people-wrap"})
northwestern_fac_dict = {}
for i in faculty_list:
    name = i.findAll("a")[1]
    link = name['href']
    name = name.text

    resp = requests.get(link)
    fac_soup = BeautifulSoup(resp.text, "html.parser")
    
    text = ""
    for word in fac_soup:
        text += word.text
        
    northwestern_fac_dict[(name, link)] = text
    

In [29]:
print_sorted_faculty(northwestern_fac_dict, False)

Found and scored 29 faculty
Joshua Leonard http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/leonard-josh.html 42
Danielle Tullman-Ercek http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/tullman-ercek-danielle.html 22
Neha Kamat http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/kamat-neha.html 12
Brian Hoffman https://www.chemistry.northwestern.edu/people/core-faculty/profiles/brian-hoffman.html 11
Adilson Motter http://www.physics.northwestern.edu/people/faculty/core-faculty/adilson-motter.html 10
Julius Lucks http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/lucks-julius.html 8
Heather Pinkett http://www.molbiosci.northwestern.edu/people/core-faculty/heather-pinkett.html 8
Curt Horvath http://www.molbiosci.northwestern.edu/people/core-faculty/curt-horvath.html 7
Michael Jewett http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/jewett-mike.html 7
Richard Morimoto http